# 📘 Descrição do Notebook

Este notebook realiza a **extração, tratamento, armazenamento e validação de dados** da **Pokémon API**, com foco nos Pokémons **Charmander**, **Charmeleon** e **Charizard**.

As principais etapas deste processo são:

---

## 🔹 1. Extração de Dados
- Utilização da biblioteca `requests` para acessar a API pública da PokéAPI;
- Requisição de informações dos Pokémons selecionados por meio de suas URLs individuais.

## 🔹 2. Verificação de Respostas
- Checagem do código de status HTTP (`status_code`);
- Exibição de alertas personalizados caso ocorra falha na obtenção de dados.

## 🔹 3. Criação de Tabelas Individuais (DataFrames)
Para cada Pokémon, são extraídos e organizados os seguintes dados:
- **Nome**
- **Experiência Base**
- **Altura**
- **Peso**
- **ID**

## 🔹 4. Unificação e Tratamento de Dados
- Junção dos DataFrames em uma única tabela;
- Padronização dos nomes das colunas com `str.title()`;
- Preenchimento de valores ausentes com `fillna(0)`;
- Conversão dos tipos de dados para `float` quando necessário;
- Padronização dos nomes dos Pokémons com `str.title()`.

## 🔹 5. Visualização Final
- Exibição da tabela final tratada com `display()` para facilitar a leitura.

## 🔹 6. Armazenamento em Banco de Dados
- Utilização do `sqlite3` para criar e conectar ao banco local `pokemons.db`;
- Salvamento da tabela no banco com o nome `tabela_pokemons`.

## 🔹 7. Validação dos Dados
- Leitura direta dos dados armazenados no banco;
- Exibição da tabela recuperada com `display()`.

## 🔹 8. Sistema de Alerta
- Verificação automática de Pokémons com **experiência base acima de 200**;
- Geração de mensagens de alerta com `print()` para destacar os casos encontrados.

---

> 💡 Este notebook demonstra, de forma prática, como integrar APIs, tratar dados com `pandas` e armazenar resultados em um banco de dados local com `sqlite3`.


In [1]:
import requests  # Biblioteca para pegar dados da internet
import pandas as pd  # Biblioteca para trabalhar com tabelas
import sqlite3  # Biblioteca para trabalhar com banco de dados SQLite

# Função para exibir alertas (simulação da função 'alerta' da aula)
def alerta(mensagem):
    print(f"🚨 Alerta: {mensagem}")  # Exibe um alerta na tela

# Aqui eu defino as URLs dos pokémons que quero pegar
url_charmander = 'https://pokeapi.co/api/v2/pokemon/charmander'
url_charmeleon = 'https://pokeapi.co/api/v2/pokemon/charmeleon'
url_charizard = 'https://pokeapi.co/api/v2/pokemon/charizard'

# Agora eu pego os dados dessas URLs usando requests.get()
print("Pegando os dados da API")  # Só para saber que começou
res_charmander = requests.get(url_charmander)
res_charmeleon = requests.get(url_charmeleon)
res_charizard = requests.get(url_charizard)

# Verifico se a resposta deu certo (código 200 significa sucesso)
if res_charmander.status_code == 200:
    dados_charmander = res_charmander.json()  # Converte para dicionário
else:
    alerta("Erro ao acessar API do Charmander!")

if res_charmeleon.status_code == 200:
    dados_charmeleon = res_charmeleon.json()
else:
    alerta("Erro ao acessar API do Charmeleon!")

if res_charizard.status_code == 200:
    dados_charizard = res_charizard.json()
else:
    alerta("Erro ao acessar API do Charizard!")

# Criando tabelas (dataframes) com os dados principais de cada Pokémon
print("Criando tabelas")  # Só para saber que está nessa parte

df_charmander = pd.DataFrame([{
    'Nome': 'Charmander',
    'Experiência Base': dados_charmander['base_experience'],
    'Altura': dados_charmander['height'],
    'Peso': dados_charmander['weight'],
    'ID': dados_charmander['id']
}])

df_charmeleon = pd.DataFrame([{
    'Nome': 'Charmeleon',
    'Experiência Base': dados_charmeleon['base_experience'],
    'Altura': dados_charmeleon['height'],
    'Peso': dados_charmeleon['weight'],
    'ID': dados_charmeleon['id']
}])

df_charizard = pd.DataFrame([{
    'Nome': 'Charizard',
    'Experiência Base': dados_charizard['base_experience'],
    'Altura': dados_charizard['height'],
    'Peso': dados_charizard['weight'],
    'ID': dados_charizard['id']
}])

# Agora eu junto todas as tabelas em uma só
print("Juntando as tabelas")  # De novo, só para saber onde estou
df_pokemons = pd.concat([df_charmander, df_charmeleon, df_charizard])

# Agora vem a parte de TRATAR os dados!
print("Tratando os dados")  # Essa parte é importante!

# Ajustar nomes das colunas para deixar mais bonito
df_pokemons.columns = df_pokemons.columns.str.title()  # Deixa com letra maiúscula

# Tratamento de valores ausentes (missing values)
df_pokemons = df_pokemons.fillna(0)  # Se tiver algo vazio, vira 0

# Ajustar tipos de variáveis (Altura e Peso são números)
df_pokemons['Altura'] = df_pokemons['Altura'].astype(float)
df_pokemons['Peso'] = df_pokemons['Peso'].astype(float)

# Transformação de string: deixar os nomes dos Pokémons com letra maiúscula no começo
df_pokemons['Nome'] = df_pokemons['Nome'].str.title()

# Mostrar a tabela final
print("Tabela final tratada:")
display(df_pokemons)

# ========== NOVA PARTE: Salvando os dados no banco de dados ==========
print("Salvando os dados no banco de dados SQLite...")

# Conecta ou cria o banco local
conexao = sqlite3.connect('pokemons.db')

# Salva os dados tratados na tabela
df_pokemons.to_sql('tabela_pokemons', conexao, if_exists='replace', index=False)

print("📦 Dados salvos com sucesso no banco 'pokemons.db' na tabela 'tabela_pokemons'!")

# ========== NOVA PARTE: Validando os dados salvos no banco ==========
print("Validando os dados diretamente do banco:")

# Lê os dados salvos no banco
df_validado = pd.read_sql('SELECT * FROM tabela_pokemons', conexao)

# Mostra os dados recuperados do banco
display(df_validado)

# Fecha a conexão
conexao.close()

# ========== SISTEMA DE ALERTA ==========
print("Verificando experiência base...")

for index, row in df_pokemons.iterrows():
    if row['Experiência Base'] > 200:
        alerta(f"{row['Nome']} tem mais de 200 de experiência base!")


Pegando os dados da API
Criando tabelas
Juntando as tabelas
Tratando os dados
Tabela final tratada:


,Nome,Experiência Base,Altura,Peso,Id
0,Charmander,62,6.0,85.0,4
0,Charmeleon,142,11.0,190.0,5
0,Charizard,267,17.0,905.0,6


Salvando os dados no banco de dados SQLite...
📦 Dados salvos com sucesso no banco 'pokemons.db' na tabela 'tabela_pokemons'!
Validando os dados diretamente do banco:


,Nome,Experiência Base,Altura,Peso,Id
0,Charmander,62,6.0,85.0,4
1,Charmeleon,142,11.0,190.0,5
2,Charizard,267,17.0,905.0,6


Verificando experiência base...
🚨 Alerta: Charizard tem mais de 200 de experiência base!


### 🔍 Conclusão

Este notebook demonstrou um processo completo de extração, transformação e armazenamento de dados de três Pokémons da API oficial PokéAPI: Charmander, Charmeleon e Charizard.

As principais etapas realizadas foram:

- Acesso à API com `requests` para obter os dados brutos dos Pokémons.
- Criação de tabelas com `pandas` para organizar as informações principais.
- Tratamento dos dados, incluindo padronização de nomes, tipos de variáveis e valores ausentes.
- Armazenamento dos dados tratados em um banco de dados SQLite.
- Validação final ao ler os dados diretamente do banco para garantir que foram salvos corretamente.
- Implementação de um sistema simples de alerta que notifica se algum Pokémon tem mais de 200 de experiência base.

Este projeto serviu como prática de integração de diversas ferramentas e conceitos essenciais em projetos de dados, como requisições a APIs, manipulação de dados com `pandas` e persistência com banco de dados.

Como próximos passos, seria interessante:

- Adicionar visualizações dos dados com gráficos.
- Expandir para mais Pokémons (por geração, tipo ou outra lógica).
- Criar funções reutilizáveis para automatizar o processo.

